In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations

In [3]:
df_food = pd.read_csv("df_food_filtered_with_food_group.csv",index_col=0)
with open("df_food_openfoodfacts_merged.pkl", "rb") as f:
    df_food_openfoodfacts = pickle.load(f, encoding="latin1")

C:\Users\Sophie\AppData\Local\Temp\ipykernel_20060\1585345951.py:1: DtypeWarning: Columns (2,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food = pd.read_csv("df_food_filtered_with_food_group.csv",index_col=0)


In [4]:
df_food = df_food[df_food["eaten_quantity_in_gram"] > 0]
df_food = df_food[df_food["energy_kcal_eaten"] > 0]

In [5]:
df_food_scaled = pd.read_csv('df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_20060\4074791732.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('df_food_scaled.csv')


In [6]:
df_food_scaled[['QI', 'DI','NB', 'combined_name', 'food_group']].groupby('food_group').median(numeric_only=True).sort_values('QI')

,QI,DI,NB
food_group,,,
alcoholic_beverages,0.038127,0.007668,3.812709
sweetened_beverages,0.041806,0.664300,4.180602
others,0.105016,0.666667,5.474351
soups_sauces_condiments,0.117561,1.068889,6.402525
salty_snacks,0.121828,0.535766,10.000000
cooked_grains_pasta,0.135563,0.517566,9.358974
sweets_desserts,0.157373,0.852030,12.639102
vegetable_oils,0.245871,0.694173,6.645221
butter_margarine_spreads,0.284294,1.837749,15.794434


In [7]:
df_eggs = df_food_scaled[df_food_scaled['food_group'] == 'eggs']

In [8]:
df_eggs.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
5666,"Egg, scrambled, with herbs, prepared",3.611291,3.805001,82.514542
5684,"Egg, chicken, whole, hard boiled",3.255655,4.323154,64.299480
5651,"Egg, scrambled, with mushrooms, prepared",3.200082,3.818593,74.265273
262601,"Egg, scrambled, prepared",3.029581,3.957133,66.868111
5815,"French omelette, prepared",2.982629,3.922503,66.631592
161820,Egg yolk,2.252876,4.320843,71.010584
305897,Coop NaturaFarm: 6 eggs,2.150097,4.447589,64.542604
295822,"Coop Naturaplan: Pic-Nic Eier Bio, gekocht/gef...",2.150097,4.447589,64.542604
281006,Coop Naturaplan Bio: 6 Eier gross,2.150097,4.447589,64.542604
281059,Coop Naturaplan Bio: 9 Eier,2.150097,4.447589,64.542604


There is no abberant values for the eggs